# Business Analytics problem - Predicting the viewership

#### Importing required packages

In [2]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model

#### Setting up the directory

In [5]:
os.getcwd()

'C:\\Users\\Bryan\\NBA Hackaton\\Business Analytics'

In [5]:
#Run only once
os.chdir(os.getcwd()+'\\Business Analytics\\Business Analytics\\')

In [6]:
print(os.getcwd())
print(os.listdir())

C:\Users\arvra\Documents\UVa files\NBA Analytics Competition\Business Analytics\Business Analytics
['.RData', '.Rhistory', '2018 NBA Hackathon Application - Business Track Prompt.pdf', 'game_data.csv', 'player_data.csv', 'player_data_edit.csv', 'player_data_edit.xlsx', 'test_set.csv', 'Train summary.csv', 'training_set.csv', 'us_bank_holiday.csv']


In [55]:
game_data = pd.read_csv('game_data.csv')
player_data = pd.read_csv('player_data.csv')
train = pd.read_csv('training_set.csv')
test = pd.read_csv('test_set.csv')

### Analyzing input files

In [56]:
train = train.rename(columns= {"Rounded Viewers":"Total_Viewers"})
train.head()

,Season,Game_ID,Game_Date,Away_Team,Home_Team,Country,Total_Viewers
0,2016-17,21600001,10/25/2016,NYK,CLE,C113,18
1,2016-17,21600001,10/25/2016,NYK,CLE,C193,1
2,2016-17,21600001,10/25/2016,NYK,CLE,C126,4
3,2016-17,21600001,10/25/2016,NYK,CLE,C163,11
4,2016-17,21600001,10/25/2016,NYK,CLE,C73,3


In [8]:
test.head()

,Season,Game_ID,Game_Date,Away_Team,Home_Team,Total_Viewers
0,2016-17,21600010,10/26/2016,DEN,NOP,NaN
1,2016-17,21600023,10/28/2016,HOU,DAL,NaN
2,2016-17,21600027,10/29/2016,BOS,CHA,NaN
3,2016-17,21600026,10/29/2016,ATL,PHI,NaN
4,2016-17,21600048,11/1/2016,LAL,IND,NaN


#### Appending training and test dataset to create common features across train and test

In [57]:

## Identifier to test: Country and Total_Viewers is NaN
train['dataset'] = "Train"
test['dataset'] = "Test"
train_test = train.append(test)
train_test.tail()

C:\Users\Bryan\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,Away_Team,Country,Game_Date,Game_ID,Home_Team,Season,Total_Viewers,dataset
455,CHA,NaN,4/10/2018,21701213,IND,2017-18,NaN,Test
456,UTA,NaN,4/11/2018,21701229,POR,2017-18,NaN,Test
457,HOU,NaN,4/11/2018,21701230,SAC,2017-18,NaN,Test
458,WAS,NaN,4/11/2018,21701222,ORL,2017-18,NaN,Test
459,DET,NaN,4/11/2018,21701224,CHI,2017-18,NaN,Test


In [10]:
print(train.shape)
print(test.shape)
print(train_test.shape)

(280625, 8)
(460, 7)
(281085, 8)


### Game data

In [ ]:
game_data

In [58]:
## Adding date related features
game_data['Year'] = pd.DatetimeIndex(game_data.Game_Date).year
game_data['Month'] = pd.DatetimeIndex(game_data.Game_Date).month
game_data['Weekday'] = pd.DatetimeIndex(game_data.Game_Date).dayofweek

In [59]:
game_feature = game_data.loc[~game_data.L2M_Score.isnull(),'Team':'Weekday'] 

### Player data

In [60]:
player_data.head()

,Season,Game_ID,Game_Date,Team,Person_ID,Name,ASG_Team,Active_Status,Minutes,Points,Defensive_Rebounds,Offensive_Rebounds,Assists,Steals,Blocks,Turnovers,Field_Goals,Field_Goals_Attempted,Three_Pointers,Three_Pointers_Attempted,Free_Throws,Free_Throws_Attempted,Personal_Fouls
0,2016-17,21600001,10/25/2016,CLE,101112,Channing Frye,None,Active,14.0,6.0,2.0,2.0,0.0,1.0,1.0,1.0,2.0,6.0,2.0,5.0,0.0,0.0,4.0
1,2016-17,21600001,10/25/2016,CLE,201567,Kevin Love,East ASG,Active,25.0,23.0,10.0,2.0,2.0,3.0,0.0,2.0,6.0,15.0,2.0,6.0,9.0,12.0,3.0
2,2016-17,21600001,10/25/2016,CLE,202681,Kyrie Irving,East ASG,Active,30.0,29.0,1.0,1.0,4.0,2.0,0.0,0.0,12.0,22.0,4.0,7.0,1.0,1.0,1.0
3,2016-17,21600001,10/25/2016,CLE,202684,Tristan Thompson,None,Active,22.0,0.0,4.0,2.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0
4,2016-17,21600001,10/25/2016,CLE,202697,Iman Shumpert,None,Active,14.0,2.0,1.0,1.0,3.0,0.0,1.0,2.0,1.0,3.0,0.0,1.0,0.0,0.0,2.0


In [61]:
player_data['Month'] = pd.DatetimeIndex(player_data.Game_Date).month
player_data['Weekday'] = pd.DatetimeIndex(player_data.Game_Date).dayofweek
player_data['Week'] = pd.DatetimeIndex(player_data.Game_Date).week
player_data['Year'] = pd.DatetimeIndex(player_data.Game_Date).year

In [62]:
player_data.head()

,Season,Game_ID,Game_Date,Team,Person_ID,Name,ASG_Team,Active_Status,Minutes,Points,Defensive_Rebounds,Offensive_Rebounds,Assists,Steals,Blocks,Turnovers,Field_Goals,Field_Goals_Attempted,Three_Pointers,Three_Pointers_Attempted,Free_Throws,Free_Throws_Attempted,Personal_Fouls,Month,Weekday,Week,Year
0,2016-17,21600001,10/25/2016,CLE,101112,Channing Frye,None,Active,14.0,6.0,2.0,2.0,0.0,1.0,1.0,1.0,2.0,6.0,2.0,5.0,0.0,0.0,4.0,10,1,43,2016
1,2016-17,21600001,10/25/2016,CLE,201567,Kevin Love,East ASG,Active,25.0,23.0,10.0,2.0,2.0,3.0,0.0,2.0,6.0,15.0,2.0,6.0,9.0,12.0,3.0,10,1,43,2016
2,2016-17,21600001,10/25/2016,CLE,202681,Kyrie Irving,East ASG,Active,30.0,29.0,1.0,1.0,4.0,2.0,0.0,0.0,12.0,22.0,4.0,7.0,1.0,1.0,1.0,10,1,43,2016
3,2016-17,21600001,10/25/2016,CLE,202684,Tristan Thompson,None,Active,22.0,0.0,4.0,2.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,10,1,43,2016
4,2016-17,21600001,10/25/2016,CLE,202697,Iman Shumpert,None,Active,14.0,2.0,1.0,1.0,3.0,0.0,1.0,2.0,1.0,3.0,0.0,1.0,0.0,0.0,2.0,10,1,43,2016


In [63]:
player_data['Points_per_minute'] = player_data.Points/player_data.Minutes
player_features = player_data.drop(['Game_ID','Person_ID'], axis = 1).groupby(['Year','Month','Team','Active_Status'],as_index=False).sum().query('Active_Status != "Inactive"')

In [18]:
player_features.head()

,Year,Month,Team,Active_Status,Minutes,Points,Defensive_Rebounds,Offensive_Rebounds,Assists,Steals,...,Field_Goals,Field_Goals_Attempted,Three_Pointers,Three_Pointers_Attempted,Free_Throws,Free_Throws_Attempted,Personal_Fouls,Weekday,Week,Points_per_minute
0,2016,10,ATL,Active,480.0,220.0,64.0,31.0,52.0,24.0,...,77.0,167.0,23.0,56.0,43.0,65.0,39.0,104,1690,9.649007
2,2016,10,BKN,Active,961.0,416.0,132.0,54.0,79.0,38.0,...,151.0,364.0,41.0,142.0,73.0,89.0,79.0,143,2249,18.451606
4,2016,10,BOS,Active,480.0,221.0,68.0,15.0,58.0,19.0,...,86.0,165.0,19.0,55.0,30.0,42.0,45.0,130,1677,8.895925
6,2016,10,CHA,Active,478.0,204.0,76.0,24.0,47.0,15.0,...,75.0,181.0,14.0,45.0,40.0,48.0,42.0,143,1677,8.022046
8,2016,10,CHI,Active,720.0,341.0,116.0,44.0,85.0,22.0,...,126.0,267.0,31.0,73.0,58.0,69.0,57.0,104,1690,15.777534


### Features for home and away games

#### Creating Montly features across teams to capture the Team's performance over time, which might be a good predictor for viewership

In [64]:
game_feat_group = game_feature.loc[game_feature.Location == "H",].groupby(['Team','Location','Month','Year'], as_index = False)
game_feat_home = game_feat_group.mean()
#print(game_feat_home.head())

In [65]:
game_feat_group_away = game_feature.loc[game_feature.Location == "A",].groupby(['Team','Location','Month','Year'], as_index = False)
game_feat_away = game_feat_group_away.mean()

## Working on the training dataset
#### Adding features from game dataset and creating date related features

### Holiday data
#### United State's holiday calender as feature set

In [21]:
# Country_viewers = train.groupby(['Country']).sum().sort_values(['Rounded Viewers'],ascending = False)
# Country_viewers['Percentage'] = 100 * (Country_viewers['Rounded Viewers']/sum(Country_viewers['Rounded Viewers']))
# Country_viewers

In [66]:
holiday_data = pd.read_csv("us_bank_holiday.csv",header = None)
holiday_data = holiday_data.rename(columns = {0:'Sno',1:"Date",2:"Holiday"}).drop('Sno', axis = 1)
holiday_data['Holiday_flag'] = 1
holiday_data = holiday_data.drop(['Holiday'], axis = 1)
holiday_data['Date'] = pd.DatetimeIndex(holiday_data.Date)
holiday_data.head()

,Date,Holiday_flag
0,2012-01-02,1
1,2012-01-16,1
2,2012-02-20,1
3,2012-05-28,1
4,2012-07-04,1


#### Diluting the 'Country' column since it is not a part of the test data

In [67]:
train_summ = train_test.groupby(['dataset','Season','Game_ID','Game_Date','Away_Team','Home_Team'],as_index = False).sum().rename(columns = {'Rounded Viewers':'Total_Viewers'})

In [68]:
train_summ['Date'] = pd.DatetimeIndex(train_summ.Game_Date)

In [69]:
### Merging the holiday data with the training dataset
train_summ = pd.merge(train_summ,holiday_data, on = ['Date'],how = 'left' )

In [70]:
print(train_summ.head())
#Checking if zero viewership is only available for test dataset
train_summ[train_summ.Total_Viewers == 0].dataset.value_counts()

  dataset   Season   Game_ID   Game_Date Away_Team Home_Team  Total_Viewers  \
0    Test  2016-17  21600010  10/26/2016       DEN       NOP            0.0   
1    Test  2016-17  21600023  10/28/2016       HOU       DAL            0.0   
2    Test  2016-17  21600026  10/29/2016       ATL       PHI            0.0   
3    Test  2016-17  21600027  10/29/2016       BOS       CHA            0.0   
4    Test  2016-17  21600048   11/1/2016       LAL       IND            0.0   

        Date  Holiday_flag  
0 2016-10-26           NaN  
1 2016-10-28           NaN  
2 2016-10-29           NaN  
3 2016-10-29           NaN  
4 2016-11-01           NaN  


Test    460
Name: dataset, dtype: int64

### Date features from Training data

In [71]:
train_summ.Holiday_flag = train_summ.Holiday_flag.fillna(0)

In [72]:
train_summ['Month'] = pd.DatetimeIndex(train_summ.Game_Date).month
train_summ['Year'] = pd.DatetimeIndex(train_summ.Game_Date).year
train_summ['Weekday'] = pd.DatetimeIndex(train_summ.Game_Date).weekday
train_summ['Week'] = pd.DatetimeIndex(train_summ.Game_Date).week

#### Merging Game data

In [73]:
### Merging the contents of game data
game_feat_away.drop(['Weekday'], axis = 1, inplace = True)
game_feat_home.drop(['Weekday'], axis = 1, inplace = True)
train_with_game = pd.merge(train_summ, game_feat_away,how = 'left', left_on = ["Away_Team","Month","Year"], right_on = ["Team","Month","Year"])
train_with_game = pd.merge(train_with_game, game_feat_home,how = 'left', left_on = ["Home_Team","Month","Year"], right_on = ["Team","Month","Year"])

In [74]:
train_with_game[train_with_game.Team_x.isna()].dataset.value_counts()
train_with_game[train_with_game.Team_y.isna()].dataset.value_counts()

Test    5
Name: dataset, dtype: int64

In [75]:
train_with_game.head()

,dataset,Season,Game_ID,Game_Date,Away_Team,Home_Team,Total_Viewers,Date,Holiday_flag,Month,Year,Weekday,Week,Team_x,Location_x,Wins_Entering_Gm_x,Losses_Entering_Gm_x,Team_Minutes_x,Final_Score_x,Lead_Changes_x,Ties_x,Largest_Lead_x,Full_Timeouts_x,Short_Timeouts_x,Qtr_4_Score_x,L2M_Score_x,Team_y,Location_y,Wins_Entering_Gm_y,Losses_Entering_Gm_y,Team_Minutes_y,Final_Score_y,Lead_Changes_y,Ties_y,Largest_Lead_y,Full_Timeouts_y,Short_Timeouts_y,Qtr_4_Score_y,L2M_Score_y
0,Test,2016-17,21600010,10/26/2016,DEN,NOP,0.0,2016-10-26,0.0,10,2016,2,43,DEN,A,1.0,1.000000,240.0,102.000000,4.000000,0.000000,6.000000,6.000000,1.0,84.000000,101.000000,NOP,H,0.000000,1.000000,240.000000,114.000000,1.000000,1.000000,2.000000,6.000000,1.000000,81.000000,103.000000
1,Test,2016-17,21600023,10/28/2016,HOU,DAL,0.0,2016-10-28,0.0,10,2016,4,43,HOU,A,0.0,0.000000,240.0,114.000000,4.000000,4.000000,10.000000,5.000000,1.0,96.000000,112.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Test,2016-17,21600026,10/29/2016,ATL,PHI,0.0,2016-10-29,0.0,10,2016,5,43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PHI,H,0.000000,0.000000,240.000000,97.000000,6.000000,2.000000,11.000000,5.000000,2.000000,75.000000,95.000000
3,Test,2016-17,21600027,10/29/2016,BOS,CHA,0.0,2016-10-29,0.0,10,2016,5,43,BOS,A,1.0,0.000000,240.0,99.000000,1.000000,0.000000,1.000000,5.000000,1.0,73.000000,95.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Test,2016-17,21600048,11/1/2016,LAL,IND,0.0,2016-11-01,0.0,11,2016,1,44,LAL,A,6.0,5.857143,240.0,105.571429,3.142857,2.142857,8.142857,4.428571,1.0,75.285714,99.571429,IND,H,5.428571,6.571429,243.571429,99.714286,2.857143,1.142857,16.142857,4.428571,1.142857,78.285714,93.571429


In [76]:
#### Treating the NA values with the previous month's values

def select_next_best_values(record, Team_type, Column):
    #Month = record.Month
    #print(record.dataset)
    if record.Month != 1:
        if Team_type == "Away":
            req_row = train_with_game[(train_with_game['Month'] == (record.Month - 1)) & (train_with_game['Year'] == record.Year) & (train_with_game['Away_Team'] == record.Away_Team)]
            if req_row.shape[0] == 0:
                req_row = train_with_game[(train_with_game['Month'] == (record.Month + 1)) & (train_with_game['Year'] == record.Year) & (train_with_game['Away_Team'] == record.Away_Team)]
        if Team_type == "Home":
            req_row = train_with_game[(train_with_game['Month'] == (record.Month - 1)) & (train_with_game['Year'] == record.Year) & (train_with_game['Home_Team'] == record.Home_Team)]
            if req_row.shape[0] == 0:
                req_row = train_with_game[(train_with_game['Month'] == (record.Month + 1)) & (train_with_game['Year'] == record.Year) & (train_with_game['Home_Team'] == record.Home_Team)]
    return(req_row[Column].unique()[0])

In [77]:
home_cntry = list(train_with_game.columns[train_with_game.columns.str.contains("_x")])
#home_cntry.append('Weekday')
away_cntry = train_with_game.columns[train_with_game.columns.str.contains("_y")]

In [78]:
### To be replaced with apply function for faster computation
for each in home_cntry:
    #print(each)
    train_with_game[each] = train_with_game.apply(lambda x: select_next_best_values(x,"Home",each) if pd.isna(x[each]) else x[each], axis=1)

for each in away_cntry:
    #print(each)
    train_with_game[each] = train_with_game.apply(lambda x: select_next_best_values(x,"Away",each) if pd.isna(x[each]) else x[each], axis=1)

In [79]:
train_summ = train_with_game

### Removing columns with the tag 'Location' to avoid singularity in modeling

req_cols = train_summ.columns.str.contains('Location|Team_|Active|Points_per_minute')
train_summ = train_summ.loc[:,~req_cols]


train_summ.index = train_summ.Game_ID
train_summ = train_summ.drop(['Game_Date','Game_ID'],axis = 1)

In [ ]:
X_train.head()

### Including extra features from external sources

In [80]:
largestLead = game_data[["Game_ID", "Largest_Lead"]].groupby("Game_ID").max()
game = game_data[["Game_ID", "Final_Score"]]
maxScore = game.groupby("Game_ID").max()
minScore = game.groupby("Game_ID").min()
finalScore= maxScore - minScore
totalScore = maxScore + minScore
totalScore["Total Score"] = maxScore + minScore
totalScore.drop("Final_Score",axis =1 ,inplace = True)

In [81]:
ties = game_data[["Game_ID", "Ties"]].groupby("Game_ID").sum()

leadchange = game_data[["Game_ID", "Lead_Changes"]].groupby("Game_ID").sum()

teams = pd.Series(game_data["Team"].unique(), name='Team')


twitter = pd.Series([3260000,2100000,1150000,845000,5700000,2300000,4730000,1600000,1500000,1100000,3280000,889000,1720000,863000,901000,839000,885000,941000,836000,808000,1540000,2650000,1010000,988000,2640000,7640000,1210000,870000,4120000,1380000], index=teams, name="Twitter")


favrank = pd.Series([13,12,14,23,3,5,8,28,15,18,1,30,6,26,10,24,9,27,17,25,20,7,21,29,11,2,19,22,4,16], index=teams, name='Ranking')


allstar1 = pd.Series([3,1,0,1,4,1,0,0,0,1,1,0,2,0,1,0,0,1,1,0,0,1,0,1,1,0,1,1,1,1], index=teams, name='2016-17')
allstar2 = pd.Series([2,1,1,0,4,1,1,0,0,1,2,0,2,1,1,1,2,0,2,0,1,2,0,0,1,0,0,2,0,0], index=teams, name='2017-18')
allstar = pd.concat([allstar1, allstar2], axis=1)


s1 = pd.Series([1,13,5,9,1,2,3,11,6,7,5,14,2,8,12,6,13,7,12,10,15,3,14,10,8,15,4,10,9,4], index=teams, name='2015-16')
s2 = pd.Series([2,12,8,5,1,2,9,13,11,7,1,15,3,10,6,11,13,7,10,9,14,6,15,12,3,14,5,4,8,4], index=teams, name='2016-17')
s3 = pd.Series([4,11,3,5,2,7,6,14,13,5,2,12,1,9,7,10,8,14,6,9,3,4,15,12,1,11,15,8,13,10], index=teams, name='2017-18')
seeding = pd.concat([s1, s2,s3], axis=1)


j1 = pd.Series([9,6,0,0,10,5,0,0,0,0,3,0,1,0,0,0,0,0,0,0,2,4,0,0,0,7,0,0,8,0], index=teams, name='2016-17')
j2 = pd.Series([9,3,0,0,10,0,0,0,0,0,6,0,0,0,5,0,0,0,0,0,8,4,0,0,1,7,0,0,2,0], index=teams, name='2017-18')
jersey = pd.concat([j1, j2], axis=1)

In [82]:
def totalAllStar(away,home,season):
    return allstar.loc[away,season] + allstar.loc[home,season]

def totalseed(away,home,season):
    return seeding.loc[away,season] + seeding.loc[home,season]

def combinedRank(away,home):
    return favrank[away] + favrank[home]

def totalseedPre(away,home,season):
    if season == "2016-17":
        return seeding.loc[away,"2015-16"] + seeding.loc[home,"2015-16"]
    if season == "2017-18":
        return seeding.loc[away,"2016-17"] + seeding.loc[home,"2016-17"]
def totalTwitter(away,home):
    return twitter.loc[away] + twitter.loc[home]

In [810]:
#train_summ_bk = train_summ #train_summ["Home_Team"]

In [83]:
train_summ['Total All Star'] = train_summ.apply(lambda x: totalAllStar(x["Away_Team"],x["Home_Team"],x["Season"]), axis=1)
train_summ['Combined Seed'] = train_summ.apply(lambda x: totalseed(x["Away_Team"],x["Home_Team"],x["Season"]), axis=1)
train_summ['Combined Team Ranking'] = train_summ.apply(lambda x: combinedRank(x["Away_Team"],x["Home_Team"]), axis=1)
train_summ['PRE Combined Team Ranking'] = train_summ.apply(lambda x: totalseedPre(x["Away_Team"],x["Home_Team"],x["Season"]), axis=1)
train_summ['Total Twitter Followers'] = train_summ.apply(lambda x: totalTwitter(x["Away_Team"],x["Home_Team"]), axis=1)

In [84]:
summ_model = pd.get_dummies(train_summ)
summ_model.head()

,Total_Viewers,Date,Holiday_flag,Month,Year,Weekday,Week,Wins_Entering_Gm_x,Losses_Entering_Gm_x,Final_Score_x,Lead_Changes_x,Ties_x,Largest_Lead_x,Full_Timeouts_x,Short_Timeouts_x,Qtr_4_Score_x,L2M_Score_x,Wins_Entering_Gm_y,Losses_Entering_Gm_y,Final_Score_y,Lead_Changes_y,Ties_y,Largest_Lead_y,Full_Timeouts_y,Short_Timeouts_y,Qtr_4_Score_y,L2M_Score_y,Total All Star,Combined Seed,Combined Team Ranking,PRE Combined Team Ranking,Total Twitter Followers,dataset_Test,dataset_Train,Season_2016-17,Season_2017-18,Away_Team_ATL,Away_Team_BKN,Away_Team_BOS,Away_Team_CHA,Away_Team_CHI,Away_Team_CLE,Away_Team_DAL,Away_Team_DEN,Away_Team_DET,Away_Team_GSW,Away_Team_HOU,Away_Team_IND,Away_Team_LAC,Away_Team_LAL,Away_Team_MEM,Away_Team_MIA,Away_Team_MIL,Away_Team_MIN,Away_Team_NOP,Away_Team_NYK,Away_Team_OKC,Away_Team_ORL,Away_Team_PHI,Away_Team_PHX,Away_Team_POR,Away_Team_SAC,Away_Team_SAS,Away_Team_TOR,Away_Team_UTA,Away_Team_WAS,Home_Team_ATL,Home_Team_BKN,Home_Team_BOS,Home_Team_CHA,Home_Team_CHI,Home_Team_CLE,Home_Team_DAL,Home_Team_DEN,Home_Team_DET,Home_Team_GSW,Home_Team_HOU,Home_Team_IND,Home_Team_LAC,Home_Team_LAL,Home_Team_MEM,Home_Team_MIA,Home_Team_MIL,Home_Team_MIN,Home_Team_NOP,Home_Team_NYK,Home_Team_OKC,Home_Team_ORL,Home_Team_PHI,Home_Team_PHX,Home_Team_POR,Home_Team_SAC,Home_Team_SAS,Home_Team_TOR,Home_Team_UTA,Home_Team_WAS
Game_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
21600010,0.0,2016-10-26,0.0,10,2016,2,43,1.0,1.000000,102.000000,4.000000,0.000000,6.000000,6.000000,1.000000,84.000000,101.000000,0.000000,1.000000,114.000000,1.000000,1.000000,2.000000,6.000000,1.000000,81.000000,103.000000,1,19,42,22,1644000,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
21600023,0.0,2016-10-28,0.0,10,2016,4,43,0.0,0.000000,114.000000,4.000000,4.000000,10.000000,5.000000,1.000000,96.000000,112.000000,6.333333,8.333333,102.333333,4.000000,2.333333,17.666667,4.000000,1.000000,80.333333,97.000000,1,14,26,14,4140000,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
21600026,0.0,2016-10-29,0.0,10,2016,5,43,6.0,5.000000,100.857143,3.000000,2.142857,9.857143,4.571429,0.857143,71.285714,94.428571,0.000000,0.000000,97.000000,6.000000,2.000000,11.000000,5.000000,2.000000,75.000000,95.000000,1,19,39,19,2750000,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
21600027,0.0,2016-10-29,0.0,10,2016,5,43,1.0,0.000000,99.000000,1.000000,0.000000,1.000000,5.000000,1.000000,73.000000,95.000000,5.428571,6.571429,99.714286,2.857143,1.142857,16.142857,4.428571,1.142857,78.285714,93.571429,1,12,25,11,4119000,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
21600048,0.0,2016-11-01,0.0,11,2016,1,44,6.0,5.857143,105.571429,3.142857,2.142857,8.142857,4.428571,1.000000,75.285714,99.571429,5.428571,6.571429,99.714286,2.857143,1.142857,16.142857,4.428571,1.142857,78.285714,93.571429,1,21,20,22,8740000,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [85]:
pd.options.display.max_columns = 500
summ_model.head()

,Total_Viewers,Date,Holiday_flag,Month,Year,Weekday,Week,Wins_Entering_Gm_x,Losses_Entering_Gm_x,Final_Score_x,Lead_Changes_x,Ties_x,Largest_Lead_x,Full_Timeouts_x,Short_Timeouts_x,Qtr_4_Score_x,L2M_Score_x,Wins_Entering_Gm_y,Losses_Entering_Gm_y,Final_Score_y,Lead_Changes_y,Ties_y,Largest_Lead_y,Full_Timeouts_y,Short_Timeouts_y,Qtr_4_Score_y,L2M_Score_y,Total All Star,Combined Seed,Combined Team Ranking,PRE Combined Team Ranking,Total Twitter Followers,dataset_Test,dataset_Train,Season_2016-17,Season_2017-18,Away_Team_ATL,Away_Team_BKN,Away_Team_BOS,Away_Team_CHA,Away_Team_CHI,Away_Team_CLE,Away_Team_DAL,Away_Team_DEN,Away_Team_DET,Away_Team_GSW,Away_Team_HOU,Away_Team_IND,Away_Team_LAC,Away_Team_LAL,Away_Team_MEM,Away_Team_MIA,Away_Team_MIL,Away_Team_MIN,Away_Team_NOP,Away_Team_NYK,Away_Team_OKC,Away_Team_ORL,Away_Team_PHI,Away_Team_PHX,Away_Team_POR,Away_Team_SAC,Away_Team_SAS,Away_Team_TOR,Away_Team_UTA,Away_Team_WAS,Home_Team_ATL,Home_Team_BKN,Home_Team_BOS,Home_Team_CHA,Home_Team_CHI,Home_Team_CLE,Home_Team_DAL,Home_Team_DEN,Home_Team_DET,Home_Team_GSW,Home_Team_HOU,Home_Team_IND,Home_Team_LAC,Home_Team_LAL,Home_Team_MEM,Home_Team_MIA,Home_Team_MIL,Home_Team_MIN,Home_Team_NOP,Home_Team_NYK,Home_Team_OKC,Home_Team_ORL,Home_Team_PHI,Home_Team_PHX,Home_Team_POR,Home_Team_SAC,Home_Team_SAS,Home_Team_TOR,Home_Team_UTA,Home_Team_WAS
Game_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
21600010,0.0,2016-10-26,0.0,10,2016,2,43,1.0,1.000000,102.000000,4.000000,0.000000,6.000000,6.000000,1.000000,84.000000,101.000000,0.000000,1.000000,114.000000,1.000000,1.000000,2.000000,6.000000,1.000000,81.000000,103.000000,1,19,42,22,1644000,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
21600023,0.0,2016-10-28,0.0,10,2016,4,43,0.0,0.000000,114.000000,4.000000,4.000000,10.000000,5.000000,1.000000,96.000000,112.000000,6.333333,8.333333,102.333333,4.000000,2.333333,17.666667,4.000000,1.000000,80.333333,97.000000,1,14,26,14,4140000,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
21600026,0.0,2016-10-29,0.0,10,2016,5,43,6.0,5.000000,100.857143,3.000000,2.142857,9.857143,4.571429,0.857143,71.285714,94.428571,0.000000,0.000000,97.000000,6.000000,2.000000,11.000000,5.000000,2.000000,75.000000,95.000000,1,19,39,19,2750000,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
21600027,0.0,2016-10-29,0.0,10,2016,5,43,1.0,0.000000,99.000000,1.000000,0.000000,1.000000,5.000000,1.000000,73.000000,95.000000,5.428571,6.571429,99.714286,2.857143,1.142857,16.142857,4.428571,1.142857,78.285714,93.571429,1,12,25,11,4119000,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
21600048,0.0,2016-11-01,0.0,11,2016,1,44,6.0,5.857143,105.571429,3.142857,2.142857,8.142857,4.428571,1.000000,75.285714,99.571429,5.428571,6.571429,99.714286,2.857143,1.142857,16.142857,4.428571,1.142857,78.285714,93.571429,1,21,20,22,8740000,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Separating the actual test and train dataset

In [86]:
summ_model_test = summ_model[summ_model.dataset_Test == 1]
summ_model_test['Game_ID'] = summ_model_test.index
print(summ_model_test.shape)

summ_model_train = summ_model[summ_model.dataset_Train == 1]
print(summ_model_train.shape)

C:\Users\Bryan\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


(460, 97)
(2000, 96)


### Train test split 80-20

In [87]:
X_train_all, X_test_all, y_train, y_test = train_test_split(summ_model_train,summ_model_train['Total_Viewers'],test_size = 0.2,random_state = 0)

In [88]:
X_train = X_train_all.drop(['Total_Viewers','Date','dataset_Test','dataset_Train'], axis = 1)
X_test = X_test_all.drop(['Total_Viewers','Date','dataset_Test','dataset_Train'], axis = 1)

In [99]:
X_train.head()

,Holiday_flag,Month,Year,Weekday,Week,Wins_Entering_Gm_x,Losses_Entering_Gm_x,Final_Score_x,Lead_Changes_x,Ties_x,Largest_Lead_x,Full_Timeouts_x,Short_Timeouts_x,Qtr_4_Score_x,L2M_Score_x,Wins_Entering_Gm_y,Losses_Entering_Gm_y,Final_Score_y,Lead_Changes_y,Ties_y,Largest_Lead_y,Full_Timeouts_y,Short_Timeouts_y,Qtr_4_Score_y,L2M_Score_y,Total All Star,Combined Seed,Combined Team Ranking,PRE Combined Team Ranking,Total Twitter Followers,Season_2016-17,Season_2017-18,Away_Team_ATL,Away_Team_BKN,Away_Team_BOS,Away_Team_CHA,Away_Team_CHI,Away_Team_CLE,Away_Team_DAL,Away_Team_DEN,Away_Team_DET,Away_Team_GSW,Away_Team_HOU,Away_Team_IND,Away_Team_LAC,Away_Team_LAL,Away_Team_MEM,Away_Team_MIA,Away_Team_MIL,Away_Team_MIN,Away_Team_NOP,Away_Team_NYK,Away_Team_OKC,Away_Team_ORL,Away_Team_PHI,Away_Team_PHX,Away_Team_POR,Away_Team_SAC,Away_Team_SAS,Away_Team_TOR,Away_Team_UTA,Away_Team_WAS,Home_Team_ATL,Home_Team_BKN,Home_Team_BOS,Home_Team_CHA,Home_Team_CHI,Home_Team_CLE,Home_Team_DAL,Home_Team_DEN,Home_Team_DET,Home_Team_GSW,Home_Team_HOU,Home_Team_IND,Home_Team_LAC,Home_Team_LAL,Home_Team_MEM,Home_Team_MIA,Home_Team_MIL,Home_Team_MIN,Home_Team_NOP,Home_Team_NYK,Home_Team_OKC,Home_Team_ORL,Home_Team_PHI,Home_Team_PHX,Home_Team_POR,Home_Team_SAC,Home_Team_SAS,Home_Team_TOR,Home_Team_UTA,Home_Team_WAS,SI
0,0.0,1,2017,6,4,14.000000,27.800000,97.800000,3.800000,2.200000,8.200000,5.400000,2.000000,73.800000,91.400000,31.285714,8.428571,112.000000,4.142857,3.142857,22.857143,4.428571,0.714286,86.857143,108.571429,1,13,20,8,3800000,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1.047750
1,0.0,11,2016,6,46,6.000000,5.000000,100.857143,3.000000,2.142857,9.857143,4.571429,0.857143,71.285714,94.428571,4.833333,8.166667,112.166667,4.000000,2.666667,13.000000,5.500000,2.333333,84.500000,103.333333,1,14,48,19,1653000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.058246
2,0.0,3,2018,2,11,36.200000,31.400000,108.400000,2.600000,1.000000,7.800000,5.800000,0.000000,81.200000,100.200000,20.285714,47.857143,102.285714,2.571429,2.142857,7.285714,5.857143,0.000000,73.285714,93.714286,1,21,38,19,2501000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.934212
3,0.0,12,2016,3,50,14.000000,11.888889,106.888889,4.222222,2.444444,7.000000,4.555556,1.222222,76.888889,99.222222,22.000000,3.750000,121.500000,2.250000,3.000000,25.000000,5.500000,1.750000,95.500000,112.500000,5,13,15,14,7800000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.827917
4,0.0,2,2017,6,8,16.666667,37.166667,102.000000,3.500000,3.333333,6.500000,4.833333,0.666667,75.500000,96.166667,23.333333,29.666667,96.666667,1.666667,1.333333,10.333333,4.666667,0.666667,72.333333,93.000000,1,21,31,26,1911000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.898103


### Creating Seasonlity Index from training dataset

In [90]:
Yearly_mean = X_train_all.groupby(['Year'],as_index = False).agg({'Total_Viewers':'mean'}).rename(columns = {'Total_Viewers':'Mean_Viewers'})

In [91]:
si = X_train_all.groupby(['Year','Month'],as_index = False).agg({'Total_Viewers':'mean'})
si = pd.merge(si,Yearly_mean, on = ['Year'], how = 'left')
si['SI'] = si.Total_Viewers/si.Mean_Viewers
si= si.drop(['Total_Viewers','Mean_Viewers'], axis = 1)
si.SI.plot()
print(si.head())
print(si.tail())

   Year  Month        SI
0  2016     10  1.624305
1  2016     11  1.058246
2  2016     12  0.827917
3  2017      1  1.047750
4  2017      2  0.898103
    Year  Month        SI
9   2017     12  0.964342
10  2018      1  1.075067
11  2018      2  1.028622
12  2018      3  0.934212
13  2018      4  0.936109


#### Adding the SI to train and validation data

In [92]:
X_train = pd.merge(X_train, si, how = 'left', on = ['Year','Month'])
X_test = pd.merge(X_test, si, how = 'left', on = ['Year','Month'])
summ_model_test = pd.merge(summ_model_test, si, how = 'left', on = ['Year','Month'])

## Simple linear model

In [93]:
Linear_model = linear_model.LinearRegression(normalize = True)

In [842]:
#X_train[X_train.Weekday != X_train.Weekday_x].head()

In [843]:
#X_train.columns

In [94]:
Linear_model.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=True)

In [95]:
Predicted_test = Linear_model.predict(X_test) * 0.84

#### MAPE calculation

In [96]:
np.mean(np.abs((y_test - Predicted_test)/y_test))

0.2838268116894527

## TPOT Regression

In [42]:
from tpot import TPOTRegressor
import xgboost

C:\Users\Bryan\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [49]:
X_train, X_test, y_train, y_test = train_test_split(summ_model_train.loc[:,summ_model_train.columns != 'Total_Viewers'].astype(np.float64),summ_model_train['Total_Viewers'].astype(np.float64),test_size = 0.2,random_state = 0)
tpot = TPOTRegressor()
# fit instance
%time tpot.fit(X_train, y_train)

In [97]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import ElasticNetCV, RidgeCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import OneHotEncoder, StackingEstimator

In [229]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import ElasticNetCV, RidgeCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import OneHotEncoder, StackingEstimator

#FINAL MODEL FROM TPOT GENETIC PROGRAMMING
training_features, testing_features, training_target, testing_target = \
            train_test_split(summ_model_train.drop(['Total_Viewers','Date','dataset_Test','dataset_Train'],axis=1), summ_model_train['Total_Viewers'].values, test_size=.2)


# Score on the training set was:-30175928.47307623
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=RidgeCV()),
    OneHotEncoder(minimum_fraction=0.05, sparse=False),
    StackingEstimator(estimator=ElasticNetCV(l1_ratio=0.9, tol=0.0001)),
    StackingEstimator(estimator=ElasticNetCV(l1_ratio=0.65, tol=0.001)),
    XGBRegressor(min_child_weight = 5, gamma =.9,subsample=.5,max_depth =20,learning_rate=.1)
#     GradientBoostingRegressor(alpha=0.9, learning_rate=0.1, loss="ls", max_depth=10, max_features=0.25, min_samples_leaf=9, min_samples_split=9, n_estimators=100, subsample=0.8500000000000001, verbose =1)
)

exported_pipeline.fit(training_features, training_target)

results = exported_pipeline.predict(testing_features) * .84

np.mean(np.abs((testing_target - results)/testing_target))

0.27561977145660793

In [204]:
# A parameter grid for XGBoost
params = {'min_child_weight':[4,5], 'gamma':[i/10.0 for i in range(3,6)],  'subsample':[i/10.0 for i in range(6,11)],
'colsample_bytree':[i/10.0 for i in range(6,11)], 'max_depth': [2,3,4]}

# Initialize XGB and GridSearch
xgb = XGBRegressor(nthread=-1) 

grid = GridSearchCV(xgb, params)
grid.fit(training_features, training_target)



NameError: name 'r2_score' is not defined

In [210]:
y_test = grid.best_estimator_.predict(testing_features)*.84


In [211]:
np.mean(np.abs((testing_target - y_test)/testing_target))

0.28626917128546586

#### Predict actual test

In [194]:
summ_model_test_1 = summ_model_test.drop('Date', axis = 1)
summ_model_test_1.shape

(460, 97)

In [200]:
training_features.columns

Index(['Holiday_flag', 'Month', 'Year', 'Weekday', 'Week',
       'Wins_Entering_Gm_x', 'Losses_Entering_Gm_x', 'Final_Score_x',
       'Lead_Changes_x', 'Ties_x', 'Largest_Lead_x', 'Full_Timeouts_x',
       'Short_Timeouts_x', 'Qtr_4_Score_x', 'L2M_Score_x',
       'Wins_Entering_Gm_y', 'Losses_Entering_Gm_y', 'Final_Score_y',
       'Lead_Changes_y', 'Ties_y', 'Largest_Lead_y', 'Full_Timeouts_y',
       'Short_Timeouts_y', 'Qtr_4_Score_y', 'L2M_Score_y', 'Total All Star',
       'Combined Seed', 'Combined Team Ranking', 'PRE Combined Team Ranking',
       'Total Twitter Followers', 'Season_2016-17', 'Season_2017-18',
       'Away_Team_ATL', 'Away_Team_BKN', 'Away_Team_BOS', 'Away_Team_CHA',
       'Away_Team_CHI', 'Away_Team_CLE', 'Away_Team_DAL', 'Away_Team_DEN',
       'Away_Team_DET', 'Away_Team_GSW', 'Away_Team_HOU', 'Away_Team_IND',
       'Away_Team_LAC', 'Away_Team_LAL', 'Away_Team_MEM', 'Away_Team_MIA',
       'Away_Team_MIL', 'Away_Team_MIN', 'Away_Team_NOP', 'Away_Team_NYK'

In [199]:
summ_model_test_1.columns

Index(['Total_Viewers', 'Holiday_flag', 'Month', 'Year', 'Weekday', 'Week',
       'Wins_Entering_Gm_x', 'Losses_Entering_Gm_x', 'Final_Score_x',
       'Lead_Changes_x', 'Ties_x', 'Largest_Lead_x', 'Full_Timeouts_x',
       'Short_Timeouts_x', 'Qtr_4_Score_x', 'L2M_Score_x',
       'Wins_Entering_Gm_y', 'Losses_Entering_Gm_y', 'Final_Score_y',
       'Lead_Changes_y', 'Ties_y', 'Largest_Lead_y', 'Full_Timeouts_y',
       'Short_Timeouts_y', 'Qtr_4_Score_y', 'L2M_Score_y', 'Total All Star',
       'Combined Seed', 'Combined Team Ranking', 'PRE Combined Team Ranking',
       'Total Twitter Followers', 'dataset_Test', 'dataset_Train',
       'Season_2016-17', 'Season_2017-18', 'Away_Team_ATL', 'Away_Team_BKN',
       'Away_Team_BOS', 'Away_Team_CHA', 'Away_Team_CHI', 'Away_Team_CLE',
       'Away_Team_DAL', 'Away_Team_DEN', 'Away_Team_DET', 'Away_Team_GSW',
       'Away_Team_HOU', 'Away_Team_IND', 'Away_Team_LAC', 'Away_Team_LAL',
       'Away_Team_MEM', 'Away_Team_MIA', 'Away_Team_MIL',

In [48]:
Actual_test_predictions = Linear_model.predict(summ_model_test_1[X_test.columns])

In [51]:
pwd

'C:\\Users\\Bryan\\NBA Hackaton\\Business Analytics'

In [201]:
results2 = exported_pipeline.predict(summ_model_test_1.drop(['Total_Viewers','dataset_Test','dataset_Train','Game_ID', 'SI'],axis=1)) * .84
# summ_model_test_1